## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path = path.join(getcwd(), "AES_Flash-S")
project_path

'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Security\\AES_Flash\\AES_Flash-S'

In [6]:
atprogram(project_path, verbose=2)

Building file: .././main.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\cmsis\5.0.1\CMSIS\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/nvmctrl" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hri" -I"../trustzone" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\Atmel\SAML11_DFP\1.0.91\include"  -O1 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -mcmse -MD -MP -MF "main.d" -MT"main.d" -MT"main.o"   -o "main.o" ".././main.c" 
Finished building: .././main.c
Building target: AES_Flash-S.elf
Invoking: ARM/GNU Linker : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe

0

## Data Logging

In [7]:
live_plot = True

Create a figure for the plot.

In [8]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [9]:
if live_plot:
    config_dict = {
        "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
        "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
        "plot_pins": [False, False, True, True],
        "gpio_delay_time" : 0.0007,
        "plot_pins_method": "line",
        "plot_xmax": 5,
        "fig": fig,
        "window_title": "Experiment AES Flash",
        "file_name_base": "experiment_aes_flash"
    }
else:
    config_dict = {
        "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
        "loggers": [LOGGER_OBJECT, LOGGER_CSV],
        "gpio_delay_time" : 0.0007,
        "file_name_base": "experiment_aes_flash"
    }

Stop criteria to pass to the logger:

In [10]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [11]:
data = []
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()

with DGILibExtra(**config_dict) as dgilib:
    dgilib.device_reset()
    dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [12]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [13]:
aes_charge, aes_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [14]:
flash_charge, flash_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [15]:
aes_encrypt_charge = aes_charge[0::2]
aes_decrypt_charge = aes_charge[1::2]
aes_encrypt_time = aes_time[0::2]
aes_decrypt_time = aes_time[1::2]

In [16]:
flash_write_charge = flash_charge[0::2]
flash_read_charge = flash_charge[1::2]
flash_write_time = flash_time[0::2]
flash_read_time = flash_time[1::2]

In [17]:
MBEDTLS_AES_BLOCK_SIZE = 16
MIN_AES_BLOCKS = 1
num_bytes = range(MIN_AES_BLOCKS * MBEDTLS_AES_BLOCK_SIZE, MBEDTLS_AES_BLOCK_SIZE * (MIN_AES_BLOCKS + len(aes_encrypt_charge)), MBEDTLS_AES_BLOCK_SIZE)
print(f"MAX_AES_BLOCKS: {MIN_AES_BLOCKS + len(aes_encrypt_charge) - 1}")

MAX_AES_BLOCKS: 100


In [18]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [19]:
results = []
for y in [aes_encrypt_charge, aes_decrypt_charge, aes_encrypt_time, aes_decrypt_time, flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 100
    # variables        = 2
    chi-square         = 5.4808e-14
    reduced chi-square = 5.5927e-16
    Akaike info crit   = -3510.01052
    Bayesian info crit = -3504.80018
[[Variables]]
    slope:      3.2286e-08 +/- 5.1204e-12 (0.02%) (init = 0)
    intercept: -4.9491e-09 +/- 4.7655e-09 (96.29%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.868

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 100
    # variables        = 2
    chi-square         = 4.3119e-14
    reduced chi-square = 4.3999e-16
    Akaike info crit   = -3533.99833
    Bayesian info crit = -3528.78799
[[Variables]]
    slope:      3.3269e-08 +/- 4.5396e-12 (0.01%) (init = 0)
    intercept:  1.1202e-10 +/- 4.2264e-09 (3772.97%) (init = 1)
[[Correlations]] (unr

In [20]:
fig2 = plt.figure(figsize=(9, 8))
fig2.canvas.set_window_title("Analysis AES Flash")

In [21]:
charge_color = 'r'
time_color = 'b'

In [22]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [23]:
lines = []
lines += ax1.plot(num_bytes, aes_encrypt_charge, charge_color+'-', label='AES Encrypt Charge')
lines += ax1.plot(num_bytes, aes_decrypt_charge, charge_color+'--', label='AES Decrypt Charge')
lines += ax2.plot(num_bytes, aes_encrypt_time, time_color+'-', label='AES Encrypt Time')
lines += ax2.plot(num_bytes, aes_decrypt_time, time_color+'--', label='AES Decrypt Time')
lines += ax1.plot(num_bytes, flash_write_charge, charge_color+'-.', label='Flash Write Charge')
lines += ax1.plot(num_bytes, flash_read_charge, charge_color+':', label='Flash Read Charge')
lines += ax2.plot(num_bytes, flash_write_time, time_color+'-.', label='Flash Write Time')
lines += ax2.plot(num_bytes, flash_read_time, time_color+':', label='Flash Read Time')
ax1.legend(handles=lines)
# [aes_encrypt_charge, aes_decrypt_charge, aes_encrypt_time, aes_decrypt_time, flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
ax1.set_title(f"AES Encrypt Charge: Base {results[0].params['intercept'].value:.03}mC plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"AES Decrypt Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"AES Encrypt Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"AES Decrypt Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n" +
             f"Flash Write Charge: Base {results[4].params['intercept'].value:.03} C plus {results[4].params['slope'].value:.03} mC per byte\n" +
             f"Flash Read Charge: Base {results[5].params['intercept'].value:.03} C plus {results[5].params['slope'].value:.03} mC per byte\n" +
             f"Flash Write Time: Base {results[6].params['intercept'].value:.03} s plus {results[6].params['slope'].value:.03} s per byte\n" +
             f"Flash Read Time: Base {results[7].params['intercept'].value:.03} s plus {results[7].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [24]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(aes_encrypt_charge, open("aes_flash_encrypt_charge.p", "wb"))
pickle.dump(aes_decrypt_charge, open("aes_flash_decrypt_charge.p", "wb"))
pickle.dump(flash_write_charge, open("aes_flash_write_charge.p", "wb"))
pickle.dump(flash_read_charge, open("aes_flash_read_charge.p", "wb"))

In [8]:
%%writefile experiment_aes_flash.py

from os import getcwd, path
import pickle

from pydgilib_extra import *
from atprogram.atprogram import atprogram

def experiment_aes_flash(project_root=getcwd(), verbose=1):
    project_path = path.join(project_root, "AES_Flash-S")
    atprogram(project_path, verbose=verbose)

    config_dict = {
        "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
        "loggers": [LOGGER_OBJECT, LOGGER_CSV],
        "gpio_delay_time" : 0.0007,
        "file_name_base": "experiment_aes_flash",
        "log_folder": project_root,
        "verbose": verbose-1
    }
    
    def stop_fn(logger_data):
        return all(logger_data.gpio.values[-1])
    
    if verbose:
        print(f"Start DGILibExtra with config: \n{config_dict}\n")

    data = []
    with DGILibExtra(**config_dict) as dgilib:
        dgilib.device_reset()
        dgilib.logger.log(1000,stop_fn)
        data = dgilib.data
        
    if verbose:
        print(f"DGILibExtra data: {data}")

    def stop_function(pin_values):
        return all(pin_values)

    aes_charge, aes_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)
    flash_charge, flash_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

    aes_encrypt_charge = aes_charge[0::2]
    aes_decrypt_charge = aes_charge[1::2]
    aes_encrypt_time = aes_time[0::2]
    aes_decrypt_time = aes_time[1::2]

    flash_write_charge = flash_charge[0::2]
    flash_read_charge = flash_charge[1::2]
    flash_write_time = flash_time[0::2]
    flash_read_time = flash_time[1::2]

    # Save Charge amount list into pickle file
    pickle.dump(aes_encrypt_charge, open(path.join(project_root, "aes_flash_encrypt_charge.p"), "wb"))
    pickle.dump(aes_decrypt_charge, open(path.join(project_root, "aes_flash_decrypt_charge.p"), "wb"))
    pickle.dump(flash_write_charge, open(path.join(project_root, "aes_flash_write_charge.p"), "wb"))
    pickle.dump(flash_read_charge, open(path.join(project_root, "aes_flash_read_charge.p"), "wb"))
    
    if verbose:
        print(f"Saved results in: {path.join(project_root)}")

if __name__ == "__main__":
   experiment_aes_flash()

Overwriting experiment_aes_flash.py
